In [64]:
from PIL import Image
import numpy as np
import math
from scipy import signal
import cv2

# Part 2

---



## Question 1

---



In [65]:
def boxfilter(n) :
  # Assert that the input is odd
  assert n % 2
  # Return a 2D array representing the boxfilter with values of 1/n^2
  return np.full((n, n), 1 / (n * n))


### Tests

---



In [66]:
boxfilter(3)

array([[0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111]])

In [67]:
boxfilter(4)

AssertionError: 

In [ ]:
boxfilter(5)

## Question 2

---



In [ ]:
def gauss1d(sigma) :
  # Get the next odd integer of 6 times sigma rounded up
  len_ = math.ceil(sigma * 6)
  if len_ % 2 == 0 :
    len_ += 1

  # Create an array of length from above that contains integers representing distance from center
  k = len_ // 2
  arr = np.arange(-k,k+1)

  # Apply the Gaussian function to each entry of the array
  f = lambda x : np.exp(-(x ** 2) / (2 * (sigma ** 2)))
  arr = f(arr)

  # Normalize the array so that the sum is one
  arr /= sum(arr)
  return arr.copy()

### Tests

---



In [ ]:
print(gauss1d(0.3))

In [ ]:
print(gauss1d(0.5))

In [ ]:
print(gauss1d(1))

In [ ]:
print(gauss1d(2))

## Question 3

---



In [ ]:
def gauss2d(sigma) :
  # Generate 1D Gaussian Array
  gauss_1d = gauss1d(sigma)

  # Make 2D arrays of the previous 1D filter and its Transpose
  gauss_1d_ = gauss_1d[:,np.newaxis]
  gauss_1d_transpose = gauss_1d[np.newaxis, :]

  # Compute the convolution
  convolution = signal.convolve2d(gauss_1d_, gauss_1d_transpose)
  return convolution

### Tests

---



In [ ]:
print(gauss2d(0.5))

In [ ]:
print(gauss2d(1))

## Question 4

---



### (a)

---



In [ ]:
def convolve2d_manual(image :np.ndarray, filter:np.ndarray) :

  # Calculate the k value
  f_size = len(filter)
  k = f_size // 2

  image = np.array(image, dtype=np.float32)
  filter = np.array(filter, dtype=np.float32)

  # Normalize the image, keep track of image sum
  sum_ = np.sum(image, dtype=np.float32)
  image /= sum_


  # Add zero padding for the top/bottom, left/right k most elements
  image_padded = np.pad(image, ((k,k),(k,k)), 'constant') # array with padding
  convolution = np.zeros_like(image_padded, dtype=np.float32) # resulting convolution
  filter = np.flipud(np.fliplr(filter)) # since it's convolution we the original filter by 180 degrees
  flat_filter = np.array(filter).flatten() # flattening the filter for computation efficiency

  # Convolution process
  for Y in range(k, len(image_padded) - k) :
    for X in range(k, len(image_padded[Y]) - k) :
      neighbor = image_padded[Y - k: Y + k + 1, X - k: X + k + 1].flatten()
      # sum of F(i,j) * I(X + i, Y + j)
      convolution[Y, X] = np.sum(neighbor * flat_filter)

  # Remove zero padding from before
  convolution = convolution[k : -k, k : -k]

  # Undo normalization
  convolution *= sum_
  return convolution.copy()

### Test case

---



In [ ]:
convolve2d_manual([[1,0,0,0,1],[2,3,0,8,0],[2,0,0,0,3],[0,0,1,0,0]],[[1/9,1/9,1/9],[1/9,1/9,1/9],[1/9,1/9,1/9]])

### (b)

---



In [ ]:
def gaussconvolve2d_manual(array, sigma):
  # 2D Gaussian filter from above
  filter = gauss2d(sigma)

  # Return Convolution result (manual)
  return convolve2d_manual(array, filter)

### Test

---



In [ ]:
gaussconvolve2d_manual([[1,0,0,0,1],[2,3,0,8,0],[2,0,0,0,3],[0,0,1,0,0]], 0.6)

### (c)

---



In [ ]:
# Convert image to grey scale
img = Image.open('./drive/MyDrive/images/dog.jpg').convert('L')

# Convert image into array of grey scale values
img_arr = np.asarray(img,dtype=np.float32)

# Filter the image via convolution with sigma value of 3
filtered_img_arr = gaussconvolve2d_manual(img_arr, 3)

# Ensure array values are within 0 ~ 255 and type uint8
filtered_img_arr_uint8 = np.clip(filtered_img_arr, 0, 255).astype(np.uint8)

### (d)

---



In [ ]:
# Convert array back to Image
filtered_img = Image.fromarray(filtered_img_arr_uint8)

# Image Comparison between original image and filtered image
print("Original Image:")
display(img)
print("Filtered Image:")
display(filtered_img)

## Question 5

---



### (a)

---



In [ ]:
def gaussconvolve2d_scipy(array, sigma):
  # 2D Gaussian filter from above
  filter = gauss2d(sigma)

  # Return Convolution Result (scipy)
  return signal.convolve2d(array, filter, 'same').copy()

The reason there is both the `signal.convolve2d` and `signal.correlate2d` is because they iterate through the filter in different directions.

The reason `gauss2d` produces the same result for both is that because the Gaussian Filter is a symmetric filter. Rotating the Gaussian Filter by 180 degrees would be the exact same filter, so it wouldn't matter.

If the filter is not symmetric, hence when rotated by 180 degrees gives us a different filter, the the results will be different for correlation and convolution.

### (b)

---



In [ ]:
# Convert image to array
img_arr = np.asarray(img)

# Apply convolution with 2D Gaussian Filter
filtered_img_arr = gaussconvolve2d_scipy(img_arr, 3)

# Ensure array contains uint8 within 0 ~ 255
filtered_img_arr_uint8 = np.clip(filtered_img_arr, 0, 255).astype(np.uint8)

### (c)

---



In [ ]:
# Convert array back to image
filtered_img = Image.fromarray(filtered_img_arr_uint8)

# Comparison of images
print("Original Image:")
display(img)
print("Filtered Image:")
display(filtered_img)

## Question 6

---



In [ ]:
import time
# Convert Image to grey scale
dog = Image.open('./drive/MyDrive/images/dog.jpg').convert('L')

# Convert image to array
img_arr = np.asarray(dog, dtype=np.float32)

# Record time before running manual convolution
t1 = time.time()

# Run convolution (manual)
manual_img = gaussconvolve2d_manual(img_arr, 10)

# Compute time duration for manual convolution
duration_manual = time.time() - t1

# Record time before running scipy convolution
t2 = time.time()

# Run convolution (scipy)
scipy_img = gaussconvolve2d_scipy(img_arr, 10)

# Compute time duration for scipy convolution
duration_scipy = time.time() - t2

# Time comparison
print(f'manual time: {duration_manual}, scipy time: {duration_scipy}')
print(f'Time difference (manual relative to scipy) : {duration_manual - duration_scipy}')

### Comment

---




By comparing the time duration of both my implementation and the scipy implementation, though the difference varies we can always see that the scipy implementation is more time efficient.

Considering the Github repository for scipy. https://github.com/scipy/scipy we can see that the convolve2d would use the fastest implementation possible, such as fft (fast fourier transformation) and employing separability.

Hence, considering that our implementation is neither fft or using separability, scipy's implementation would be faster.

## **Question 7**

---



Convolution with a 2D Gaussian filter can be made more efficient by using the fact that Gaussian filters are separable. Instead of performing a 2D convolution directly, we can decompose it into two consecutive 1D convolutions: one applied horizontally and the other vertically. Which would reduce the time complexity reduces the complexity from $O(n^2 \cdot m^2)$ to $O(n^2 \cdot m)$ where $m$ is the filter dimension and $n$ is the image dimension.



# Part 3

---



## Helper Functions I made

---



In [ ]:
# Helper function for splitting image into RGB channel arrays
def split_img_rgb(filename) :
  # Open image file
  img = Image.open(f'./drive/MyDrive/images/{filename}')

  # Convert to RGB scale
  img = img.convert("RGB")

  # Split RGB Channels
  img_r, img_g, img_b = img.split()

  # Convert each channel to array
  img_r = np.asarray(img_r, dtype=np.float32)
  img_g = np.asarray(img_g, dtype=np.float32)
  img_b = np.asarray(img_b, dtype=np.float32)

  # Return original image and each channel array
  return img, img_r, img_g, img_b

# Helper function to apply 2D Gaussian filtering to each RGB channel
def img_gauss2d_convolution(img_r, img_g, img_b, sigma) :
  # Apply 2D gaussian convolution for each channel
  img_r_cv = gaussconvolve2d_scipy(img_r, sigma)
  img_g_cv = gaussconvolve2d_scipy(img_g, sigma)
  img_b_cv = gaussconvolve2d_scipy(img_b, sigma)

  # Return each channel after filtering
  return img_r_cv, img_g_cv, img_b_cv

# Helper function to convert rgb channel arrays to one image
def array_to_img(img_r, img_g, img_b, offset=0) :

  # Assure that each channel is within 0 ~ 255 and in uint8 format
  # Offset is used for images with zero-mean
  img_r = np.clip(img_r + offset, 0, 255).astype(np.uint8)
  img_g = np.clip(img_g + offset, 0, 255).astype(np.uint8)
  img_b = np.clip(img_b + offset, 0, 255).astype(np.uint8)

  # Convert each channel to image from array
  img_r = Image.fromarray(img_r)
  img_g = Image.fromarray(img_g)
  img_b = Image.fromarray(img_b)

  # Merge to one image
  img = Image.merge("RGB", (img_r, img_g, img_b))

  return img

# Helper function to create hybrid images
def hybrid_image(img_low, img_high, sigma):

  # Split low pass image into RGB channels and apply convolution
  img, img_low_r, img_low_g, img_low_b = split_img_rgb(img_low)
  img_low_r, img_low_g, img_low_b = img_gauss2d_convolution(img_low_r, img_low_g, img_low_b, sigma)

  # Split high pass image into RGB channels and apply convolution, get high pass frequencies
  img_, img_high_r, img_high_g, img_high_b = split_img_rgb(img_high)
  img_high_r_cv, img_high_g_cv, img_high_b_cv = img_gauss2d_convolution(img_high_r, img_high_g, img_high_b, sigma)

  img_high_r -= img_high_r_cv
  img_high_g -= img_high_g_cv
  img_high_b -= img_high_b_cv

  # Make Hybrid RGB channels
  img_hybrid_r = img_high_r + img_low_r
  img_hybrid_g = img_high_g + img_low_g
  img_hybrid_b = img_high_b + img_low_b

  # Convert Hybrid RGB channels to image
  hybrid_img = array_to_img(img_hybrid_r, img_hybrid_g, img_hybrid_b)
  return hybrid_img

## (1)

---



In [ ]:
# sigma value selected
sigma = 7

# split image into each channel along with getting image
img, img_r, img_g, img_b = split_img_rgb('0b_dog.bmp')

# get low frequency channels via convolution
img_r_cv_low_f, img_g_cv_low_f, img_b_cv_low_f = img_gauss2d_convolution(img_r, img_g, img_b, sigma)

# merge each channel to get low frequency image
low_frequency_img = array_to_img(img_r_cv_low_f, img_g_cv_low_f, img_b_cv_low_f)

# display original image and low frequency image
print("Original Image:")
display(img)
print("Low Frequency Image:")
display(low_frequency_img)

## (2)

---



In [ ]:
# split image into each channel along with getting image
img, img_r, img_g, img_b = split_img_rgb('0a_cat.bmp')

# get lower frequency rgb channels
filtered_img_r_cv, filtered_img_g_cv, filtered_img_b_cv = img_gauss2d_convolution(img_r, img_g, img_b, sigma)

# obtain higher frequency channels by subtracting lower frequency channels from original channels
img_r_cv_high_f = img_r - filtered_img_r_cv
img_g_cv_high_f = img_g - filtered_img_g_cv
img_b_cv_high_f = img_b - filtered_img_b_cv

# obtain high frequency image from above, add 128 to visualize
high_frequency_img = array_to_img(img_r_cv_high_f, img_g_cv_high_f, img_b_cv_high_f, 128)

# display original image and high
print("Original Image:")
display(img)
print("High Frequency Image:")
display(high_frequency_img)

## (3)

---



In [ ]:
# Create Hybrid rgb channels by combining high / low frequencies
hybrid_img_r_cv = img_r_cv_low_f + img_r_cv_high_f
hybrid_img_g_cv = img_g_cv_low_f + img_g_cv_high_f
hybrid_img_b_cv = img_b_cv_low_f + img_b_cv_high_f

# Create Hybrid image from rgb channels
hybrid_image = array_to_img(hybrid_img_r_cv, hybrid_img_g_cv, hybrid_img_b_cv)

# Display Image
print("Hybrid Image:")
display(hybrid_image)

### Tests

---



#### First Set

In [ ]:
image_1 = hybrid_image('1a_bicycle.bmp','1b_motorcycle.bmp',5)
display(image_1)
image_2 = hybrid_image('1a_bicycle.bmp','1b_motorcycle.bmp',7)
display(image_2)
image_3 = hybrid_image('1a_bicycle.bmp','1b_motorcycle.bmp',9)
display(image_3)

#### Second set

In [ ]:
image_1 = hybrid_image('2a_einstein.bmp','2b_marilyn.bmp',6)
display(image_1)
image_2 = hybrid_image('2a_einstein.bmp','2b_marilyn.bmp',8)
display(image_2)
image_3 = hybrid_image('2a_einstein.bmp','2b_marilyn.bmp',10)
display(image_3)


#### Third Set

In [ ]:
image_1 = hybrid_image('3a_fish.bmp','3b_submarine.bmp',9)
display(image_1)
image_2 = hybrid_image('3a_fish.bmp','3b_submarine.bmp',11)
display(image_2)
image_3 = hybrid_image('3a_fish.bmp','3b_submarine.bmp',13)
display(image_3)

## Part 4

---



### (1)

---



In [ ]:
# box_gauss image
box_gauss = cv2.imread('drive/MyDrive/images/box_gauss.png')

# parameters for Gaussiablur
ksize = 13
sigmaX = 100
gauss = cv2.GaussianBlur(box_gauss, ksize=(ksize, ksize), sigmaX=sigmaX)

# Image display
display(gauss)

# parameters for median blur
ksize_median = 9
median = cv2.medianBlur(box_gauss,ksize=ksize_median)

# Image display
display(median)

# parameters for bilateral filter
d = 13
sigmaColor = 210
sigmaSpace = 200
bilateral = cv2.bilateralFilter(box_gauss, d, sigmaColor=sigmaColor, sigmaSpace=sigmaSpace)

# Image display
display(bilateral)



In [ ]:
# box_speckle image
box_speckle = cv2.imread('drive/MyDrive/images/box_speckle.png')

# parameters for Gaussiablur
ksize = 13
sigmaX = 200
gauss = cv2.GaussianBlur(box_gauss, ksize=(ksize, ksize), sigmaX=sigmaX)

# Image display
display(gauss)

# parameters for median blur
ksize_median = 11
median = cv2.medianBlur(box_gauss,ksize=ksize_median)\

# Image display
display(median)

# parameters for bilateral filter
d = 13
sigmaColor = 210
sigmaSpace = 200
bilateral = cv2.bilateralFilter(box_gauss, d, sigmaColor=sigmaColor, sigmaSpace=sigmaSpace)

# Image display
display(bilateral)

### (2)

---



In [ ]:
# box_gauss image
box_gauss = cv2.imread('drive/MyDrive/images/box_gauss.png')

# Blur filters
gauss = cv2.GaussianBlur(box_gauss, ksize=(7, 7), sigmaX=50)
bilateral = cv2.bilateralFilter(box_gauss, 7, sigmaColor=150, sigmaSpace=150)
median = cv2.medianBlur(box_gauss,7)

# Image display
display(gauss)
display(bilateral)
display(median)

# box_speckle image
box_speckle = cv2.imread('drive/MyDrive/images/box_speckle.png')

# Blur filters
gauss2 = cv2.GaussianBlur(box_speckle, ksize=(7, 7), sigmaX=50)
bilateral2 = cv2.bilateralFilter(box_speckle, 7, sigmaColor=150, sigmaSpace=150)
median2 = cv2.medianBlur(box_speckle,7)

# Image display
display(gauss2)
display(bilateral2)
display(median2)

# **Pros and Cons of each filter**

## **Gaussian Filter** :
### **Pros** :
Noises are smoothed out by averaging pixel values, which can lead to reduction in random fluctuations in intensity.
The filter provides a consistent blurring effect across the image, which is ideal where uniform reduction in detail is desired.
### **Cons** :
Since there is no differentiation between noise and important features, it smooths out both equally. This can lead to blurring of fine details.

## **Bilateral Filter** :
### **Pros** :
The bilateral filter effectively preserves edges while reducing noise by considering both spatial closeness and pixel intensity differences. This dual consideration allows it to smooth flat regions without blurring important edges, making it particularly useful for images where maintaining sharp boundaries is essential.
### **Cons** :
For cases where noise appears as abrupt changes in pixel intensity, the bilateral filter can preserve this noise due to its nature of maintaining image details and edges. This means that while it effectively retains important structures, it may also unintentionally keep certain types of high-frequency noise (as seen in the results above).

## **Median Filter** :
### **Pros** :
Since it replaces pixel values with the median of the surrounding neighbor, it is effective in removing impulsive noises. It preserves the edges better than the Gaussian Filter, because abrupt changes in intensity are less likely to average out.
### **Cons** :
Can lead distorted shapes (as shown in the edges of the middle rectangle in the figures). And since the median of the neighbor is used as the pixel value finer details are less likely preserved comparing to the bilateral filter .